In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
warnings.simplefilter("ignore", category=RuntimeWarning)

import random
import os
from juno_classes import *

from scipy.integrate import trapz

import matplotlib.colors
from matplotlib.cm import ScalarMappable



def extract_base_exponent_scientific(number):
    # Calculate exponent with respect to base 10
    exponent = int(np.log10(number))
    base = number / (10 ** exponent)
    return base, exponent

In [ ]:
startOrbit = 8
endOrbit = 52
save_pickles = True

for i in range(startOrbit, endOrbit+1):
    print('grabbing mean mag orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/jss_reduced_inbound_1s_means_4min_pickles/reduced_orbit_' +str(i) +'_4min_means_1s.pkl'
    picklefile = open(filename,'rb')
    orbit_means = pickle.load(picklefile)
    
    orbit_means_columns = list(orbit_means.columns)
    
    
    fits_df_orbit = fits_data[orbit_means.index[0].isoformat(): orbit_means.index[-1].isoformat()]
    
    fits_bad_columns = list(['Tacc', '  "R"', ' "LAT"', 'ELON', ' "LT"',  'dr[MQ16]',
       'dr[O2+]',  'dr[S+]',  'dr[S3+]',
        '     "dn"', '  "u"', ' "du"',  '   "dT"',
       'kappa', 'dkappa', ' "uphi"', 'duphi', '   "ur"', '  "dur"', '   "uz"',
       '  "duz"', 'ISSUES1', 'ISSUES2', 'mode' ])
    
    
    bad_columns = orbit_means_columns + fits_bad_columns
    
    
    combined_df = pd.concat([orbit_means, fits_df_orbit], axis=1)
    
    combined_df = combined_df.drop(columns=bad_columns)
    
    print(combined_df)
    


    
 
    print('Doing delta dens...')
    delta_density_df = delta_densities(combined_df)
    
    if save_pickles:
        delta_density_df.to_pickle('/home/mtfranciscovich/Juno-codes/fits_delta_densities/delta_dens_10min_orbit_' +str(i)+'.pkl')

In [ ]:
def delta_densities(fits_df,window_size = 10):
    
    amu = 1.67*10**-27
    
    m16 = 24*amu
    mo2 = 16*amu
    ms = 32*amu
    ms3 = 32*amu
    
    n_raw = fits_df['      "n"']
    m16_raw = fits_df['r[MQ16]']
    mo_raw = fits_df['r[O2+]']
    ms_raw = fits_df['r[S+]']
    ms3_raw = fits_df['r[S3+]']

    mean_density_data = pd.DataFrame({'MEAN_n': [], 'MEAN_mass': [], 'MEAN_density': [], 'MEAN_temperature': []})
    finish_datetime = (fits_df.index[-1]
    - timedelta(minutes=np.floor(window_size/2)))
    for datetime_index in fits_df.index:
        start_datetime = datetime_index
        end_datetime = start_datetime + timedelta(minutes=window_size)
        mean_datetime_index = pd.DatetimeIndex([
            (start_datetime + timedelta(minutes=round(window_size/2))).isoformat()
        ])
        
        check_array = n_raw[start_datetime.isoformat(): end_datetime.isoformat()].dropna()
        
        if len(check_array) < 4:
            
            continue
        temp_mean_n = np.nanmean(n_raw[start_datetime.isoformat(): end_datetime.isoformat()])
        temp_mean_m16 = np.nanmean(m16_raw[start_datetime.isoformat(): end_datetime.isoformat()])*m16
        temp_mean_mo = np.nanmean(mo_raw[start_datetime.isoformat(): end_datetime.isoformat()])*mo2
        temp_mean_ms = np.nanmean(ms_raw[start_datetime.isoformat(): end_datetime.isoformat()])*ms
        temp_mean_ms3 = np.nanmean(ms3_raw[start_datetime.isoformat(): end_datetime.isoformat()])*ms3
        
        temp_mean_mass = temp_mean_m16+temp_mean_mo+temp_mean_ms+temp_mean_ms3
        temp_mean_density = temp_mean_n*temp_mean_mass
        
        temp_mean_temperature = np.nanmean(fits_df['    "T"'][start_datetime.isoformat(): end_datetime.isoformat()])


        mean_array_to_concat = pd.DataFrame(
            {'MEAN_n': temp_mean_n,
                'MEAN_mass': temp_mean_mass,
                'MEAN_density': temp_mean_density,
            'MEAN_temperature': temp_mean_temperature},
            
            index=mean_datetime_index)
        mean_density_data = pd.concat([mean_density_data, mean_array_to_concat])
        
        if mean_datetime_index == finish_datetime:
            break
    # mean_mag_data and data_df are cut to align the time series of each
    # mag_data loses half of the time_window in the front
    # mean_mag_data loses half of the time window in the end
    # The two dataframes are then concatenated into one for simplicity
    #data_df = data_df.drop(data_df[: (data_df.index[0] +
    #     timedelta(minutes=round(window_size / 2) - 1)
    # ).isoformat()].index)
    




    fits_df = fits_df[mean_density_data.index[0].isoformat(): 
                                mean_density_data.index[-1].isoformat()]
    fits_df = pd.concat([fits_df, mean_density_data], axis=1)
    
    print('density_mean_df made, looking at perts...')
    del mean_density_data
    

    # The perturbation components of the mean field are found.
    # The method used is described in Khurana & Kivelson 1989
    
    
    #mean_mag_vecs = mag_data_df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
    #mean_mag_magnitude = np.sqrt((mean_mag_vecs**2).sum(axis=1))
    
    
    
    mean_density = fits_df['MEAN_density']
    raw_density = n_raw* ( m16_raw*m16+mo_raw*mo2+ms_raw*ms + ms3_raw*ms3)

    
    perturbed_density = raw_density - mean_density

    
    perturbed_density_df = pd.DataFrame({'density_pert': perturbed_density})

    

            
    data_df = pd.concat([fits_df, perturbed_density_df], axis=1)

    return data_df

    


In [ ]:
def mean_field_align_v_new(mag_data_df, v_data_df, window_size=10):
    v_data_df = v_data_df.dropna()
    """Rotate magnetometer data into a mean-field-aligned coordinate system.
        Using the methods described by Khurana & Kivelson[1989]

    Parameters
    ----------
    window_size : int, optional
        The size of the window in minutes that is moved over data to average over.
            This should be EVEN to ensure times of MFA and regular data line up.
            The default is 24.

    Returns
    -------
    None.

    """
    # A windows of size 'window_size' in minutes is then moved along the data
    # An average inside of the window is found for each entry
    #print(self.data_df)
    mean_v_data = pd.DataFrame({'MEAN_UX': [], 'MEAN_UY': [], 'MEAN_UZ': []})
    finish_datetime = (v_data_df.index[-1]
    - timedelta(minutes=np.floor(window_size/2)))
    for datetime_index in v_data_df.index:
        start_datetime = datetime_index
        end_datetime = start_datetime + timedelta(minutes=window_size)
        mean_datetime_index = pd.DatetimeIndex([
            (start_datetime + timedelta(minutes=round(window_size/2))).isoformat()
        ])
        
        check_array = v_data_df[start_datetime.isoformat(): end_datetime.isoformat()].dropna()
        
        if len(check_array) < 4:
            
            continue
        
        
        temp_mean_x = np.nanmean(v_data_df[start_datetime.isoformat(): end_datetime.isoformat()].UX)
        temp_mean_y = np.nanmean(v_data_df[start_datetime.isoformat(): end_datetime.isoformat()].UY)
        temp_mean_z = np.nanmean(v_data_df[start_datetime.isoformat(): end_datetime.isoformat()].UZ)

        mean_array_to_concat = pd.DataFrame(
            {'MEAN_UX': temp_mean_x,
                'MEAN_UY': temp_mean_y,
                'MEAN_UZ': temp_mean_z}, index=mean_datetime_index)
        mean_v_data = pd.concat([mean_v_data, mean_array_to_concat])
        
        if mean_datetime_index == finish_datetime:
            break
    # mean_mag_data and data_df are cut to align the time series of each
    # mag_data loses half of the time_window in the front
    # mean_mag_data loses half of the time window in the end
    # The two dataframes are then concatenated into one for simplicity
    #data_df = data_df.drop(data_df[: (data_df.index[0] +
    #     timedelta(minutes=round(window_size / 2) - 1)
    # ).isoformat()].index)
    




    v_data_df = v_data_df[mean_v_data.index[0].isoformat(): 
                                mean_v_data.index[-1].isoformat()]
    v_data_df = pd.concat([v_data_df, mean_v_data], axis=1)
    
    print('v_mean_df made, looking at perts...')
    del mean_v_data
    

    # The perturbation components of the mean field are found.
    # The method used is described in Khurana & Kivelson 1989
    
    
    #mean_mag_vecs = mag_data_df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
    #mean_mag_magnitude = np.sqrt((mean_mag_vecs**2).sum(axis=1))
    
    mean_mag_x = mag_data_df['MEAN_BX']
    mean_mag_y = mag_data_df['MEAN_BY']
    mean_mag_z = mag_data_df['MEAN_BZ']
    mean_mag_magnitude = np.sqrt(mean_mag_x**2+mean_mag_y**2+mean_mag_z**2)
    mean_mag_vecs = pd.DataFrame({'MEAN_BX': mean_mag_x, 'MEAN_BY': mean_mag_y,'MEAN_BZ': mean_mag_z}, index = mean_mag_x.index)
    
    mean_v_vecs = v_data_df[['MEAN_UX', 'MEAN_UY', 'MEAN_UZ']]
    mean_v_magnitude = np.sqrt((mean_v_vecs**2).sum(axis=1))
    
    mean_ux = v_data_df['MEAN_UX']
    mean_uy = v_data_df['MEAN_UY']
    mean_uz = v_data_df['MEAN_UZ']
    raw_ux = v_data_df['UX']
    raw_uy = v_data_df['UY']
    raw_uz = v_data_df['UZ']
    
    perturbed_ux = raw_ux - mean_ux
    perturbed_uy = raw_uy - mean_uy
    perturbed_uz = raw_uz - mean_uz
    
    perturbed_v_df = pd.DataFrame({'u_x_pert': perturbed_ux, 'u_y_pert': perturbed_uy, 'u_z_pert': perturbed_uz})
    perturbed_v_df_perp = pd.DataFrame({'u_x_pert_perp': [], 'u_y_pert_perp': [], 'u_z_pert_perp': []})
    perturbed_v_df_par = pd.DataFrame({'u_x_pert_par': [], 'u_y_pert_par': [], 'u_z_pert_par': []})
    par_mag_v_df = pd.DataFrame({'u_pert_par': []})
    
    for i in range(len(perturbed_ux)):
        new_df_index = pd.DatetimeIndex([v_data_df.index[i].isoformat()])
        

        
        try: 
            x_for_hat = mean_mag_vecs['MEAN_BX'].loc[new_df_index]
            y_for_hat = mean_mag_vecs['MEAN_BY'].loc[new_df_index]
            z_for_hat = mean_mag_vecs['MEAN_BZ'].loc[new_df_index]

            parallel_mag_hat = (np.asarray([x_for_hat,y_for_hat,z_for_hat]) / mean_mag_magnitude.loc[new_df_index][0])

            
            perturbed_vparallel_to_mag = np.dot((perturbed_v_df.iloc[i].to_numpy()), parallel_mag_hat)
            perturbed_vparallel_to_mag_vector = np.asarray(perturbed_vparallel_to_mag*parallel_mag_hat)
            perturbed_par_ux = perturbed_vparallel_to_mag_vector[0]
            perturbed_par_uy = perturbed_vparallel_to_mag_vector[1]
            perturbed_par_uz = perturbed_vparallel_to_mag_vector[2]
            perturbed_perp_ux = perturbed_v_df.iloc[i]['u_x_pert'] - perturbed_par_ux
            perturbed_perp_uy = perturbed_v_df.iloc[i]['u_y_pert'] - perturbed_par_uy
            perturbed_perp_uz = perturbed_v_df.iloc[i]['u_z_pert'] - perturbed_par_uz
            
            
            perturbed_v_df_perp = pd.concat([perturbed_v_df_perp,pd.DataFrame({'u_x_pert_perp': perturbed_perp_ux, 'u_y_pert_perp': perturbed_perp_uy, 'u_z_pert_perp': perturbed_perp_uz},
                                    index=new_df_index)])

            perturbed_v_df_par = pd.concat([perturbed_v_df_par, pd.DataFrame({'u_x_pert_par': perturbed_par_ux, 'u_y_pert_par': perturbed_par_uy, 'u_z_pert_par': perturbed_par_uz},
                            index=new_df_index) ])
            
            par_mag_v_df = pd.concat([par_mag_v_df,         pd.DataFrame({'u_pert_par': perturbed_vparallel_to_mag},
                            index=new_df_index)])
        except:
            continue
            
    data_df = pd.concat([v_data_df, perturbed_v_df_perp], axis=1)
    data_df  = pd.concat([data_df, perturbed_v_df_par], axis=1)
    data_df = pd.concat([data_df, par_mag_v_df], axis = 1)
    return data_df

    


In [ ]:
fits_data = pd.read_csv('/data/juno_spacecraft/data/FWD_FITS/JADE-I_forward_model_ion_v1/JADE-I_forward_model_ion_all_v1.csv', delimiter= ',')
def convert_datetime(datetime_str):
    # Remove the microseconds part
    datetime_str_cleaned = datetime_str.split('.')[0]
    # Convert to pandas datetime object
    date_time = pd.to_datetime(datetime_str_cleaned, format='%Y-%jT%H:%M:%S')
    # Format to the desired output
    return pd.to_datetime(date_time.strftime('%Y-%m-%dT%H:%M:%S'))

fits_data['DATETIME'] = fits_data['                "UTC"'].apply(convert_datetime)
fits_data = fits_data.drop(columns='                "UTC"')

fits_data = fits_data.set_index('DATETIME', drop=True)

ur = fits_data['   "ur"']
uphi = fits_data[' "uphi"']
uz = fits_data['   "uz"']

In [ ]:
#For above to work correctly, ie match up, need to make a df that has all
#of the date times of an orbit, like the og mag mean arrays


startOrbit = 26
endOrbit = 54
save_pickles = False

for i in range(startOrbit, endOrbit+1):
    orbit = i
    print('grabbing orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles_unit_z_mag/reduced_orbit_' +str(orbit)+'_1s.pkl'
    picklefile = open(filename,'rb')
    rawdata = pickle.load(picklefile)
    
    x = rawdata['X_JSS']
    y = rawdata['Y_JSS']
    z = rawdata['Z_JSS']
    
    Rj_km = 7.14e4
    
    

    phi = np.arctan2(y,x)

    ux = ur*np.cos(phi) - uphi*np.sin(phi)
    uy = ur*np.sin(phi) + uphi*np.cos(phi)

    orbitStartTime = x.index[0]
    orbitEndTime = x.index[-1]

    fits_data_cart_orbit = pd.DataFrame({'UX': ux[orbitStartTime:orbitEndTime], 'UY': uy[orbitStartTime:orbitEndTime], 'UZ': uz[orbitStartTime:orbitEndTime]})
    fits_data_cart_orbit['Z_MAG'] = rawdata['Z_MAG']
    fits_data_cart_orbit['R'] = np.sqrt(rawdata['X_JSS']**2 + rawdata['Y_JSS']**2)/Rj_km
    if save_pickles:
        fits_data_cart_orbit.to_pickle('/home/mtfranciscovich/Juno-codes/fits_u_cart_pickles_updated/u_cart_orbit_' +str(i)+'.pkl')
        


In [ ]:
startOrbit = 35
endOrbit = 54
save_pickles = True
mu_0 = np.pi*4e-7
gamma = 5/3
k_B = 1.38e-23

#window_size = 30

for i in range(startOrbit, endOrbit+1):
    print('grabbing mean mag orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/jss_reduced_inbound_1s_means_4min_pickles/reduced_orbit_' +str(i) +'_4min_means_1s.pkl'
    picklefile = open(filename,'rb')
    mean_mag_data = pickle.load(picklefile)
    
    orbit_means = mean_mag_data
    
    
    print('grabbing mean v orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/test_fits_u_cart_means_4min/u_cart_means_4min_orbit_' +str(i) +'.pkl'
    picklefile = open(filename,'rb')
    mean_v_data = pickle.load(picklefile)
    
    all_dates = orbit_means.index.union(fits_data.index)

    reindexed_orbit_means = orbit_means.reindex(all_dates)
    reindexed_fits_data = fits_data.reindex(all_dates)

    combined_current = reindexed_orbit_means.combine_first(reindexed_fits_data)
    
    

    beginning_of_orbit_data = orbit_means.index[0]
    end_of_orbit_data = orbit_means.index[-1]


    combined_updated_for_orbit = combined_current[beginning_of_orbit_data:end_of_orbit_data]


    print('grabbing delta densities')
    filename = '/home/mtfranciscovich/Juno-codes/code/delta_density_df.pkl'
    picklefile = open(filename,'rb')
    delta_density_data = pickle.load(picklefile)
    
    print('Calculating ratios...')
    

    ux_pert_perp = mean_v_data['u_x_pert_perp']*1e3
    uy_pert_perp = mean_v_data['u_y_pert_perp']*1e3
    uz_pert_perp = mean_v_data['u_z_pert_perp']*1e3
    
    delta_v_perp_mag = np.sqrt(ux_pert_perp**2 + uy_pert_perp**2 + uz_pert_perp**2)
    

    
    Bx_total = mean_mag_data['MEAN_BX']*(10**-9)
    By_total = mean_mag_data['MEAN_BY']*(10**-9)
    Bz_total = mean_mag_data['MEAN_BZ']*(10**-9)
    
    Bx_pert_perp = mean_mag_data['B_x_pert_perp']*(10**-9)
    By_pert_perp = mean_mag_data['B_y_pert_perp']*(10**-9)
    Bz_pert_perp = mean_mag_data['B_z_pert_perp']*(10**-9)
    
    B_pert_perp_mag = np.sqrt(Bx_pert_perp**2 + By_pert_perp**2 + Bz_pert_perp**2)
    
    Bx_pert_par = mean_mag_data['B_x_pert_par']*(10**-9)
    By_pert_par = mean_mag_data['B_y_pert_par']*(10**-9)
    Bz_pert_par = mean_mag_data['B_z_pert_par']*(10**-9)
    
    B_pert_par_mag = np.sqrt(Bx_pert_par**2 + By_pert_par**2 + Bz_pert_par**2)
    
    mean_vecs = mean_mag_data[['MEAN_BX','MEAN_BY','MEAN_BZ']]
    mean_magnitude  = np.sqrt(Bx_total**2 + By_total**2 + Bz_total**2)
    


    

    

    mean_density = delta_density_data['MEAN_density']*1e6
    mean_mass = delta_density_data['MEAN_mass']
    
    mean_temp = delta_density_data['MEAN_temperature']
    
    
    

    v_Alfven= (mean_magnitude)/np.sqrt(mu_0*mean_density) #does v_alfven need to be a vector quantity?]\
      
        
    v_S_squared =    gamma*mean_temp*11606*k_B/mean_mass
    
    delta_density = delta_density_data['density_pert']*1e6
    
    delta_density = delta_density + v_Alfven*0
    v_S_squared = v_S_squared + v_Alfven*0
    
    mean_density = mean_density + v_Alfven*0
    
    delta_v_perp_mag = delta_v_perp_mag + v_Alfven*0+mean_magnitude*0+mean_density*0
    
    compression_df = pd.DataFrame({'delta_density':delta_density, 'v_Alfven':v_Alfven, 'v_S':np.sqrt(v_S_squared), 'delta_v_perp_mag':delta_v_perp_mag, 'MEAN_density':mean_density, 'B_pert_perp_mag':B_pert_perp_mag, 'B_pert_par_mag':B_pert_par_mag, 'MEAN_B':mean_magnitude})
    
 


    
    
    
    
    print('Combine with z_mag  and R array ...')
    
    filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles/reduced_orbit_' +str(i) +'_1s.pkl'
    picklefile = open(filename,'rb')
    mag_data_mag_frame = pickle.load(picklefile)
    z_mag_info = mag_data_mag_frame['Z_MAG']
    R_info = mag_data_mag_frame['R']
    
    compression_df['Z_MAG'] = z_mag_info
    compression_df['R'] = R_info
    

    if save_pickles:
        compression_df.to_pickle('/home/mtfranciscovich/Juno-codes/compression_dfs_by_orbit/compression_df_orbit_' + str(i) + '.pkl')

    

In [ ]:
startOrbit = 36
endOrbit = 54
save_pickles = True
mu_0 = np.pi*4e-7

window_size = 30

print('grabbing mean density')
filename = '/home/mtfranciscovich/Juno-codes/code/delta_density_df_2.pkl'
picklefile = open(filename,'rb')
mean_density_data = pickle.load(picklefile)

for i in range(startOrbit, endOrbit+1):
    if i ==49:
        continue
    print('grabbing mean mag orbit ' + str(i))
    #filename = '/home/mtfranciscovich/Juno-codes/jss_downsampled_mag_pickles_10min/reduced_orbit_' +str(i) +'_10min_means_1s.pkl'
    filename = '/home/mtfranciscovich/Juno-codes/means_10min_reduced_inbound_1s_pickles_unit_z_mag_jss_downsampled/reduced_orbit_'+str(i)+'_1s.pkl'
    picklefile = open(filename,'rb')
    mean_mag_data = pickle.load(picklefile)
    
    orbit_means = mean_mag_data
    
    print('grabbing total mag orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles_unit_z_mag/reduced_orbit_'+str(i)+'_1s.pkl'
    picklefile = open(filename,'rb')
    total_mag_data = pickle.load(picklefile)
    
    
    #filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_means_4min_pickles/reduced_orbit_' + str(i) + '_4min_means_1s.pkl'
    
    #picklefile = open(filename,'rb')
    #mean_mag_data_mag_coord = pickle.load(picklefile)
    
    #orbit_means_mag_cord = mean_mag_data_mag_coord
    
    
    
    print('grabbing mean v orbit ' + str(i))
    #filename = '/home/mtfranciscovich/Juno-codes/test_fits_u_cart_means_4min/u_cart_means_4min_orbit_' +str(i) +'.pkl'
    filename = '/home/mtfranciscovich/Juno-codes/updated_fits_u_cart_means_10min/u_cart_means_10min_orbit_' +str(i)+'.pkl'
    picklefile = open(filename,'rb')
    mean_v_data = pickle.load(picklefile)
    
    print('grabbing total v orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/fits_u_cart_pickles_updated/u_cart_orbit_'+str(i)+'.pkl'
    picklefile = open(filename,'rb')
    total_v_data = pickle.load(picklefile)
    

 
    
    all_dates = orbit_means.index.union(fits_data.index)

    reindexed_orbit_means = orbit_means.reindex(all_dates)
    reindexed_fits_data = fits_data.reindex(all_dates)

    combined_current = reindexed_orbit_means.combine_first(reindexed_fits_data)
    
    

    beginning_of_orbit_data = orbit_means.index[0]
    end_of_orbit_data = orbit_means.index[-1]
    
    
    print('cleaning mean density')
    
    delta_density_data = mean_density_data[beginning_of_orbit_data:end_of_orbit_data]


    combined_updated_for_orbit = combined_current[beginning_of_orbit_data:end_of_orbit_data]

    
    print('Calculating dot prod and helicity...')
    

    ux_pert_perp = mean_v_data['u_x_pert_perp']*1e3
    uy_pert_perp = mean_v_data['u_y_pert_perp']*1e3
    uz_pert_perp = mean_v_data['u_z_pert_perp']*1e3
    
    Bx_pert_perp = mean_mag_data['B_x_pert_perp']*(10**-9)
    By_pert_perp = mean_mag_data['B_y_pert_perp']*(10**-9)
    Bz_pert_perp = mean_mag_data['B_z_pert_perp']*(10**-9)
    
    Bx_total = mean_mag_data['MEAN_BX']*(10**-9)
    By_total = mean_mag_data['MEAN_BY']*(10**-9)
    Bz_total = mean_mag_data['MEAN_BZ']*(10**-9)
    
    
    #Bx_mag_coord = mean_mag_data_mag_coord['MEAN_BX']*(10**-9)
    #By_mag_coord = mean_mag_data_mag_coord['MEAN_BY']*(10**-9)
    #Bz_mag_coord = mean_mag_data_mag_coord['MEAN_BZ']*(10**-9)
    
    mean_magnitude_mag_coord = np.sqrt(Bx_total**2+By_total**2+Bz_total**2)
    
    unitz_x = total_mag_data['UNITZ_X']
    unitz_y = total_mag_data['UNITZ_Y']
    unitz_z = total_mag_data['UNITZ_Z']
 
    
    cos_theta = (Bx_total*unitz_x+By_total*unitz_y+Bz_total*unitz_z)/mean_magnitude_mag_coord
    
    

    
    mean_vecs = mean_mag_data[['MEAN_BX','MEAN_BY','MEAN_BZ']]
    mean_magnitude  = np.sqrt(Bx_total**2 + By_total**2 + Bz_total**2)
    
    #B0 = np.dot((mean_vecs).to_numpy(), parallel_hat)
    
    B_tot_mag = np.sqrt(Bx_total**2 + By_total**2 + Bz_total**2)
    
    #test = Bx_total/B_tot_mag
    
    mean_density = delta_density_data['MEAN_density']*1e6
    mean_mass = delta_density_data['MEAN_mass']
    
    mean_temp = delta_density_data['MEAN_temperature']
    
    
    

    v_Alfven= (mean_magnitude)/np.sqrt(mu_0*mean_density)
    
    
    '''
    n = combined_updated_for_orbit['      "n"'] 
    m16 = combined_updated_for_orbit['r[MQ16]']
    mo = combined_updated_for_orbit['r[O2+]']
    ms = combined_updated_for_orbit['r[S+]']
    ms3 = combined_updated_for_orbit['r[S3+]']
    
    m16_factor = 5.32*10**-26 + 2.65*10**-26
    mo2_factor = 5.31*10**-26
    ms_factor = 5.32*10**-26
    ms3_factor = 5.32*10**-26
    

    m = m16*m16_factor + mo*mo2_factor +ms*ms_factor+ms3*ms3_factor

    density = m * n*1e6
    
    '''

    '''
    v_Alfven_x = (Bx_total)/np.sqrt(mu_0*density) #does v_alfven need to be a vector quantity?
    v_Alfven_y = (By_total)/np.sqrt(mu_0*density) #does v_alfven need to be a vector quantity?
    v_Alfven_z = (Bz_total)/np.sqrt(mu_0*density) #does v_alfven need to be a vector quantity?
    '''
    
    Bx_pert_perp_norm = Bx_pert_perp/mean_magnitude
    By_pert_perp_norm = By_pert_perp/mean_magnitude
    Bz_pert_perp_norm = Bz_pert_perp/mean_magnitude
    
    ux_pert_perp_norm = ux_pert_perp/v_Alfven
    uy_pert_perp_norm = uy_pert_perp/v_Alfven
    uz_pert_perp_norm = uz_pert_perp/v_Alfven

    

    
    
    dot_prod_for_calc = ux_pert_perp*Bx_pert_perp + uy_pert_perp*By_pert_perp + uz_pert_perp*Bz_pert_perp
    
    
    factor = np.sqrt(Bx_total**2 + By_total**2 + Bz_total**2)*(1/mu_0)
    
    delta_dot_prod = (-1)*(dot_prod_for_calc)*(factor)
    
    Helicity = dot_prod_for_calc/2
    
    norm_Helicity_num = ux_pert_perp_norm*Bx_pert_perp_norm + uy_pert_perp_norm*By_pert_perp_norm + uz_pert_perp_norm*Bz_pert_perp_norm

    
    
    delta_vperp_squared_norm = ux_pert_perp_norm**2+uy_pert_perp_norm**2+uz_pert_perp_norm**2
    delta_Bperp_squared_norm = Bx_pert_perp_norm**2+By_pert_perp_norm**2+Bz_pert_perp_norm**2
    
    delta_energy_norm = 0.5*(delta_vperp_squared_norm+delta_Bperp_squared_norm)
    
    norm_Helicity = norm_Helicity_num/delta_energy_norm
    
    
    residual_energy_num = delta_vperp_squared_norm - delta_Bperp_squared_norm
    
    residual_energy_denom = delta_vperp_squared_norm + delta_Bperp_squared_norm
    
    residual_energy_Norm = residual_energy_num/residual_energy_denom
    
    delta_vperp_magnitude = np.sqrt(ux_pert_perp**2+uy_pert_perp**2+uz_pert_perp**2)
    
    delta_Bperp_magnitude = np.sqrt(Bx_pert_perp**2+By_pert_perp**2+Bz_pert_perp**2)
    
    delta_v_alfven = delta_Bperp_magnitude/np.sqrt(mu_0*(delta_density_data['density_pert']*1e6))
    
    vx_jss = total_v_data['UX']*1e3
    vy_jss = total_v_data['UY']*1e3
    vz_jss = total_v_data['UZ']*1e3
    
    Bx_jss = total_mag_data['BX_JSS']*1e-9
    By_jss = total_mag_data['BY_JSS']*1e-9
    Bz_jss = total_mag_data['BZ_JSS']*1e-9
    
    
    v_cross_b_x = (vy_jss*Bz_jss - By_jss*vz_jss)
    v_cross_b_y = -(vx_jss*Bz_jss - Bx_jss*vz_jss)
    v_cross_b_z = (vx_jss*By_jss - Bx_jss*vy_jss)
    
    Sx_jss = (-(v_cross_b_y*Bz_jss - By_jss*v_cross_b_z))/mu_0
    Sy_jss = (v_cross_b_x*Bz_jss - Bx_jss*v_cross_b_z)/mu_0
    Sz_jss = -(v_cross_b_x*By_jss - Bx_jss*v_cross_b_y)/mu_0
    
    
    x = total_mag_data['X_JSS']
    y = total_mag_data['Y_JSS']
    
    phi = np.arctan2(y,x)
    
    S_r = Sx_jss*np.cos(phi) + Sy_jss*np.sin(phi)
    S_phi = -np.sin(phi)*Sx_jss + Sy_jss*np.cos(phi)
    

    
    S_dot_zmag = Sx_jss*unitz_x+Sy_jss*unitz_y+Sz_jss*unitz_z
    
    energy_df = pd.DataFrame({'delta_dot_prod': delta_dot_prod , 'Helicity': Helicity, 'delta_energy': delta_energy_norm, 'norm_Helicity': norm_Helicity, 'norm_residual_energy': residual_energy_Norm, 'v_Alfven': v_Alfven, 'cos_theta':cos_theta, 'delta_vperp_squared_norm': delta_vperp_squared_norm, 'delta_Bperp_squared_norm': delta_Bperp_squared_norm, 'norm_helicity_num': norm_Helicity_num,'S_r':S_r, 'S_phi':S_phi,'S_z':Sz_jss,'S_dot_zmag':S_dot_zmag,'phi':phi})  
    print('Combine with z_mag  and R array ...')
    
    #filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles/reduced_orbit_' +str(i) +'_1s.pkl'
    #picklefile = open(filename,'rb')
    #mag_data_mag_frame = pickle.load(picklefile)
    z_mag_info = total_mag_data['Z_MAG']
    Rj_km = 7.14e4
    R_info = np.sqrt(total_mag_data['X_JSS']**2+total_mag_data['Y_JSS']**2)/Rj_km    
    energy_df['Z_MAG'] = z_mag_info
    energy_df['R'] = R_info
    
    energy_df = energy_df.dropna()
    mean_energy_data = pd.DataFrame({'MEAN_delta_dot_prod': [], 'MEAN_Helicity': [], 'MEAN_delta_energy': [], 'MEAN_norm_Helicity': [], 'MEAN_norm_residual_energy': [], 'MEAN_cos_theta': [], 'residual_energy_test': [], 'norm_helicity_test': [], 'MEAN_S_r':[], 'MEAN_S_phi':[], 'MEAN_S_z':[], 'MEAN_S_dot_zmag':[]})

    finish_datetime = (energy_df.index[-1]
- timedelta(minutes=np.floor(window_size/2)))
    for datetime_index in energy_df.index:
        start_datetime = datetime_index
        end_datetime = start_datetime + timedelta(minutes=window_size)
        mean_datetime_index = pd.DatetimeIndex([
        (start_datetime + timedelta(minutes=round(window_size/2))).isoformat()
        ])
        temp_mean_delta_dot_prod = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].delta_dot_prod)
        temp_mean_Helicity = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].Helicity)
        temp_mean_delta_energy = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].delta_energy)
        temp_mean_norm_Helicity = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].norm_Helicity)
        temp_mean_norm_residual_energy = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].norm_residual_energy)
        temp_mean_cos_theta = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].cos_theta)
        
        
        temp_mean_vperp_squared_norm = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].delta_vperp_squared_norm)
        temp_mean_Bperp_squared_norm = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].delta_Bperp_squared_norm)
        
        temp_res_energy_num = temp_mean_vperp_squared_norm - temp_mean_Bperp_squared_norm
        temp_res_energy_denom = temp_mean_vperp_squared_norm + temp_mean_Bperp_squared_norm
        temp_mean_residual_energy_test = temp_res_energy_num/temp_res_energy_denom
        
        temp_mean_norm_hel_num = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].norm_helicity_num)
        temp_mean_norm_helicity_test = temp_mean_norm_hel_num/temp_res_energy_denom
        temp_mean_Bperp_squared_norm = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].delta_Bperp_squared_norm)
        temp_mean_S_r = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].S_r)
        temp_mean_S_phi = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].S_phi)
        temp_mean_S_z = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].S_z)
        temp_mean_S_dot_zmag = np.nanmean(energy_df[start_datetime.isoformat(): end_datetime.isoformat()].S_dot_zmag)
        
        mean_array_to_concat = pd.DataFrame(
        {'MEAN_delta_dot_prod': temp_mean_delta_dot_prod,
            'MEAN_Helicity': temp_mean_Helicity,
            'MEAN_delta_energy' : temp_mean_delta_energy,
            'MEAN_norm_Helicity' : temp_mean_norm_Helicity,
            'MEAN_norm_residual_energy': temp_mean_norm_residual_energy,
            'MEAN_cos_theta': temp_mean_cos_theta,
            'residual_energy_test': temp_mean_residual_energy_test,
            'norm_helicity_test': temp_mean_norm_helicity_test,
            'MEAN_S_r': temp_mean_S_r,
            'MEAN_S_phi': temp_mean_S_phi,
            'MEAN_S_z': temp_mean_S_z,
            'MEAN_S_dot_zmag': temp_mean_S_dot_zmag
            
            }, index=mean_datetime_index)
        mean_energy_data = pd.concat([mean_energy_data, mean_array_to_concat])
    
        if mean_datetime_index == finish_datetime:
            break
    energy_df = energy_df[mean_energy_data.index[0].isoformat(): 
                            mean_energy_data.index[-1].isoformat()]
    energy_df = pd.concat([energy_df, mean_energy_data], axis=1)

    if save_pickles:
        energy_df.to_pickle('/home/mtfranciscovich/Juno-codes/updated_energy_pickles_0423/energy_df_orbit_' + str(i) + '.pkl')

    